In [10]:
import requests
from langchain.agents import Tool
from langchain.llms import LlamaCpp 
from langchain.agents import initialize_agent
from collections import defaultdict
import json 

api_key = "c6dfc4d92a8f972d237ef696ec87b37a"

In [11]:
def get_weather_info(city):
    """Fetches current weather information for a city using OpenWeatherMap API."""
    
    url_current = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response_current = requests.get(url_current)
    if response_current.status_code != 200:
        return "Error: Could not fetch weather data."
    data_current = response_current.json()

    response = {
        'coordinates': data_current['coord'],
        'weather': data_current['weather'][0],
        'temperature': {
            'current': data_current['main']['temp'],
            'feels_like': data_current['main']['feels_like'],
            'min': data_current['main']['temp_min'],
            'max': data_current['main']['temp_max']
        },
        'pressure': {
            'sea_level': data_current['main'].get('sea_level', data_current['main']['pressure']),
            'ground_level': data_current['main'].get('grnd_level', data_current['main']['pressure'])
        },
        'humidity': data_current['main']['humidity'],
        'visibility': data_current['visibility'],
        'wind': data_current['wind'],
        'clouds': data_current['clouds'],
        'rain': data_current.get('rain', {}),
        'dt': data_current['dt'],
        'sys': data_current['sys'],
        'timezone': data_current['timezone'],
        'id': data_current['id'],
        'name': data_current['name'],
        'cod': data_current['cod']
    }

    return json.dumps(response, indent=2)

In [7]:
def restructure_forecast(forecast_json):
    """Restructures the forecast JSON data into a nested dictionary by date and time."""
    structured_data = defaultdict(dict)
    for entry in forecast_json['list']:
        date, time = entry['dt_txt'].split()
        structured_data[date][time] = {
            'temperature': entry['main']['temp'],
            'feels_like': entry['main']['feels_like'],
            'temp_min': entry['main']['temp_min'],
            'temp_max': entry['main']['temp_max'],
            'pressure': entry['main']['pressure'],
            'humidity': entry['main']['humidity'],
            'weather': entry['weather'][0]['description'],
            'icon': entry['weather'][0]['icon'],
            'wind_speed': entry['wind']['speed'],
            'wind_deg': entry['wind']['deg'],
            'visibility': entry['visibility'],
            'pop': entry['pop'],
            'rain': entry.get('rain', {}).get('3h', 0),
            'clouds': entry['clouds']['all']
        }
    return structured_data

def get_specific_forecast(forecast_data, query):
    """
    Extracts specific forecast information based on user queries.
    
    Args:
    forecast_data (dict): The structured forecast data.
    query (str): The user's query.
    
    Returns:
    str: The relevant forecast information.
    """
    today = datetime.today()
    query = query.lower()

    if 'tomorrow' in query:
        target_date = (today + timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'day after tomorrow' in query:
        target_date = (today + timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        # Extract specific date if mentioned in the query
        try:
            target_date = datetime.strptime(query.split()[-1], '%Y-%m-%d').strftime('%Y-%m-%d')
        except ValueError:
            # Default to today's date if no specific date is mentioned
            target_date = today.strftime('%Y-%m-%d')

    if 'morning' in query:
        target_time = '09:00:00'
    elif 'afternoon' in query:
        target_time = '15:00:00'
    elif 'evening' in query:
        target_time = '18:00:00'
    elif 'night' in query:
        target_time = '21:00:00'
    else:
        # Default to the next available time if no specific time is mentioned
        target_time = '12:00:00'

    if target_date in forecast_data:
        if target_time in forecast_data[target_date]:
            forecast = forecast_data[target_date][target_time]
            return (f"Forecast for {target_date} at {target_time}:\n"
                    f"Temperature: {forecast['temperature']}°C\n"
                    f"Feels like: {forecast['feels_like']}°C\n"
                    f"Weather: {forecast['weather']}\n"
                    f"Humidity: {forecast['humidity']}%\n"
                    f"Wind: {forecast['wind_speed']} m/s, {forecast['wind_deg']}°\n"
                    f"Visibility: {forecast['visibility']} meters\n"
                    f"Probability of precipitation: {forecast['pop']*100}%\n"
                    f"Rain: {forecast['rain']} mm\n"
                    f"Clouds: {forecast['clouds']}%")
        else:
            return f"No forecast data available for {target_date} at {target_time}."
    else:
        return f"No forecast data available for {target_date}."

def get_forecast_info(city, query):
    """Fetches and processes the weather forecast based on the user's query."""
    forecast_data = get_forecast(city)
    if isinstance(forecast_data, dict):
        return get_specific_forecast(forecast_data, query)
    else:
        return forecast_data


def get_forecast(city):
    """Fetches 5-day weather forecast for a city using OpenWeatherMap API."""
    
    url_forecast = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response_forecast = requests.get(url_forecast)
    if response_forecast.status_code != 200:
        return "Error: Could not fetch forecast data."
    forecast_json = response_forecast.json()
    
    return json.dumps(restructure_forecast(forecast_json), indent=2)

In [9]:
get_forecast_info('Kalyani', 'Afternoon')

'{\n  "2024-07-02": {\n    "15:00:00": {\n      "temperature": 27.05,\n      "feels_like": 29.66,\n      "temp_min": 27.05,\n      "temp_max": 27.26,\n      "pressure": 998,\n      "humidity": 78,\n      "weather": "light rain",\n      "icon": "10n",\n      "wind_speed": 4.39,\n      "wind_deg": 242,\n      "visibility": 10000,\n      "pop": 0.83,\n      "rain": 1.3,\n      "clouds": 100\n    },\n    "18:00:00": {\n      "temperature": 27.07,\n      "feels_like": 30.2,\n      "temp_min": 27.07,\n      "temp_max": 27.13,\n      "pressure": 999,\n      "humidity": 83,\n      "weather": "light rain",\n      "icon": "10n",\n      "wind_speed": 2.67,\n      "wind_deg": 244,\n      "visibility": 10000,\n      "pop": 0.79,\n      "rain": 0.67,\n      "clouds": 100\n    },\n    "21:00:00": {\n      "temperature": 26.51,\n      "feels_like": 26.51,\n      "temp_min": 26.51,\n      "temp_max": 26.51,\n      "pressure": 999,\n      "humidity": 88,\n      "weather": "overcast clouds",\n      "icon